# Infraestrutura para Carregamento de Veículos Elétricos - Comparação entre Países Europeus



In [26]:
import pandas as pd
import pycountry

## Introdução

Portugal anunciado na comunicação social como o 4º país da Europa com mais postos de carregamento por 100km de estrada:
https://www.acea.auto/press-release/electric-cars-6-eu-countries-have-less-than-1-charger-per-100km-of-road-1-charger-in-7-is-fast/


## Metodologia



## Data

### Estradas

In [45]:
pycountry.countries.search_fuzzy('Kosovo')

[Country(alpha_2='RS', alpha_3='SRB', flag='🇷🇸', name='Serbia', numeric='688', official_name='Republic of Serbia')]

In [54]:
df_roads_eu = (
    pd.read_csv("notebooks/europe_comparison/rawdata/estat_road_if_roadsc_en.csv")
    .drop(columns=["DATAFLOW", "LAST UPDATE", "freq", "unit", "OBS_FLAG"])
    .loc[lambda _: _.tra_infr == "TOTAL"]
    .replace("UK", "GB")
    .replace("EL", "GR")
    .replace("XK", "RS")
    .assign(**{"Country": lambda _: _.geo.apply(lambda code: pycountry.countries.get(alpha_2=code).name)})
    .rename(columns={"TIME_PERIOD": "Year", "OBS_VALUE": "TotalRoadLength"})
    .sort_values(["Country", "Year"])
    .groupby("Country")
    .tail(1)
    .drop(columns=["tra_infr", "geo"])
    .rename(columns={"Year": "RoadYear", })
    .set_index("Country")
)
df_roads_eu

,RoadYear,TotalRoadLength
Country,,
Albania,2021,3581.350
Austria,2021,124608.000
Belgium,2010,153447.000
Bosnia and Herzegovina,2021,9078.000
Bulgaria,2021,19119.000
Croatia,2021,25096.000
Cyprus,2021,12969.000
Czechia,2021,129410.000
Denmark,2021,73542.000


In [65]:
df_roads = (
    pd.read_csv("notebooks/europe_comparison/rawdata/roads_wikipedia.csv")
    .set_index("Country")[["Total"]]
    .rename(columns={"Total": "TotalRoads"})
    .join(df_roads_eu, how="outer")
    .assign(**{"TotalRoads": lambda _: _.TotalRoads.mask(_.TotalRoads.isnull(), lambda i: _.TotalRoadLength)})
    .drop(columns=["TotalRoadLength", "RoadYear"])
)
df_roads

,TotalRoads
Country,
Afghanistan,34903.0
Albania,3945.0
Algeria,141000.0
American Samoa,241.0
Andorra,320.0
...,...
Vietnam,570448.0
West Bank,4686.0
World,63690567.0


In [68]:
df_chargers = (
    pd.read_csv("notebooks/europe_comparison/rawdata/evses_2022.csv")
    .set_index("Category")
    .sum(axis=1)
    .rename("Chargers")
    .to_frame()
    .join(
        df_roads, how="left"
    )

    .assign(**{"ChargersPer100km": lambda _: _.Chargers / _.TotalRoads * 100})
    .sort_values(["ChargersPer100km"], ascending=False)
)
df_chargers

,Chargers,TotalRoads,ChargersPer100km
Category,,,
Netherlands,138607.0,139000.0,99.717266
Luxembourg,2048.0,2875.0,71.234783
Portugal,7595.0,11260.0,67.451155
Belgium,33757.0,118414.0,28.507609
Denmark,16086.0,73574.0,21.863702
Austria,20092.0,126400.0,15.895570
Germany,93121.0,644480.0,14.449013
France,102989.0,1053215.0,9.778535
Italy,37299.0,487700.0,7.647939


## Fontes

### Quantidade de veículos por categoria e país

* https://alternative-fuels-observatory.ec.europa.eu/transport-mode/road/european-union-eu27/country-comparison

* Navegar até "AF Fleet (M1&N1)" -> Download CSV


